We try to classify if the human on the image carries a mask.
We use the dataset on https://www.kaggle.com/ashishjangra27/face-mask-12k-images-dataset.

In [1]:
import tensorflow as tf

print(tf.version)

<module 'tensorflow._api.v2.version' from 'C:\\Python39\\lib\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>


In [2]:
%ls
%cd face_mask_dataset

 Volume in drive C has no label.
 Volume Serial Number is 564D-750D

 Directory of c:\Users\bksie\Documents\Documents\Source_Codes\machine-learning-experiments\Image Classification\Human-Mask-Recognition

21/11/2021  00:36    <DIR>          .
21/11/2021  00:36    <DIR>          ..
21/11/2021  00:33    <DIR>          face_mask_dataset
21/11/2021  00:36            15.064 human-face-mask-recognition.ipynb
               1 File(s)         15.064 bytes
               3 Dir(s)  103.652.093.952 bytes free
c:\Users\bksie\Documents\Documents\Source_Codes\machine-learning-experiments\Image Classification\Human-Mask-Recognition\face_mask_dataset


In [3]:
import pathlib

training_data_dir = pathlib.Path("Train")
validation_data_dir = pathlib.Path("Validation")
test_data_dir = pathlib.Path("Test")

In [4]:
batch_size = 32
img_width = 256
img_height = 256

train_ds = tf.keras.utils.image_dataset_from_directory(
  training_data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  validation_data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)



Found 10000 files belonging to 2 classes.
Using 8000 files for training.
Found 800 files belonging to 2 classes.
Using 160 files for validation.


In [5]:
class_names = train_ds.class_names
print(class_names)

['WithMask', 'WithoutMask']


In [6]:
num_classes = 2

model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=(img_height, img_width, 3)))
model.add(tf.keras.layers.Rescaling(1./255))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(7,7), strides=(2,2), activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(7,7), strides=(2,2), activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(7,7), strides=(2,2), activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(7,7), strides=(2,2), activation='relu'))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(7,7), strides=(2,2), activation='relu'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 125, 125, 32)      4736      
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 60, 32)        50208     
                                                                 
 conv2d_2 (Conv2D)           (None, 27, 27, 32)        50208     
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 32)        50208     
                                                                 
 conv2d_4 (Conv2D)           (None, 3, 3, 32)          50208     
                                                                 
Total params: 205,568
Trainable params: 205,568
Non-trai

In [7]:
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=3*3*32, activation='relu'))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 125, 125, 32)      4736      
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 60, 32)        50208     
                                                                 
 conv2d_2 (Conv2D)           (None, 27, 27, 32)        50208     
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 32)        50208     
                                                                 
 conv2d_4 (Conv2D)           (None, 3, 3, 32)          50208     
                                                                 
 flatten (Flatten)           (None, 288)               0

In [11]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_ds, validation_data=val_ds, epochs=15)

Epoch 1/15
250/250 [==============================] - 23s 87ms/step - loss: 0.1881 - accuracy: 0.9415 - val_loss: 0.1792 - val_accuracy: 0.9375
Epoch 2/15
250/250 [==============================] - 21s 85ms/step - loss: 0.1281 - accuracy: 0.9561 - val_loss: 0.1298 - val_accuracy: 0.9500
Epoch 3/15
250/250 [==============================] - 24s 97ms/step - loss: 0.2018 - accuracy: 0.9227 - val_loss: 0.1375 - val_accuracy: 0.9375
Epoch 4/15
250/250 [==============================] - 21s 85ms/step - loss: 0.1341 - accuracy: 0.9480 - val_loss: 0.1669 - val_accuracy: 0.9438
Epoch 5/15
250/250 [==============================] - 24s 94ms/step - loss: 0.1118 - accuracy: 0.9589 - val_loss: 0.1359 - val_accuracy: 0.9563
Epoch 6/15
250/250 [==============================] - 22s 87ms/step - loss: 0.0886 - accuracy: 0.9663 - val_loss: 0.1397 - val_accuracy: 0.9563
Epoch 7/15
250/250 [==============================] - 21s 83ms/step - loss: 0.0964 - accuracy: 0.9649 - val_loss: 0.1063 - val_accuracy:

In [12]:
# Test
test_ds = tf.keras.utils.image_dataset_from_directory(
  test_data_dir,
  validation_split=None,
  subset=None,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

model.evaluate(test_ds)

Found 992 files belonging to 2 classes.
31/31 [==============================] - 2s 49ms/step - loss: 0.1240 - accuracy: 0.9567


[0.12399168312549591, 0.9566532373428345]

In [30]:
import numpy as np

# sample inference
img = tf.keras.utils.load_img(path="Test/WithoutMask/3238.png", target_size=(256, 256))
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array)
score = predictions[0]

print(class_names[np.argmax(score)])
print(np.max(score))

WithoutMask
0.7019725
